# PROCESAMIENTO DE LENGUAJE NATURAL

## Trabajo Practico 1
## Gustavo Uñapillco            a1624

## Consigna del desafío 1

1. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

2. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

3. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".

## Desarrollo: 

### Carga de Datos


In [379]:
#Importación de librerías
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pandas as pd

In [380]:
#Descarga de dataset
data_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
data_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [ ]:
#Instanciar el vectorizador
vectorizador = TfidfVectorizer(
)

In [382]:
#Sets de entrenamiento
X_train = vectorizador.fit_transform(data_train.data)
y_train = data_train.target

In [383]:
#Sets de testeo
X_test = vectorizador.transform(data_test.data)
y_test = data_test.target

### **1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

In [384]:
#Función para procesar las muestras aleatorias
import random

def documentos_aleatorios():
    
    #Tomamos 5 indices de manera aleatoria:
    #idxs_aleatorios = random.sample(range (X_train.shape[0]), 5) 
    #print('Los documentos aleatorios son:', idxs_aleatorios)

    #Output: Los documentos aleatorios son: [2494, 2207, 8883, 8003, 2581]
    
    #Solo por razones de prueba tomamos los siguientes valores devueltos por el codigo: 
    idxs_aleatorios=[2494, 2207, 8883, 8003, 2581]

    #Obtenemos las etiquetas originales
    etiquetas_originales = []
    for idx in idxs_aleatorios:
        etiquetas_originales.append(data_train.target_names[y_train[idx]])
    #print(etiquetas_originales)

    # Medimos la similaridad coseno de los 5 documentos con todos los documentos de train
    similarity_ls = []
    for idx in idxs_aleatorios:
        cossim = cosine_similarity(X_train[idx], X_train)[0]
        similarity_ls.append(cossim)
    
    #Ordenamos todos los arrays de la lista similarity_ls
    similarity_ord_ls = []
    similarity_idx_ls = []

    for cossim in similarity_ls:
        similarity_ord_ls.append(np.sort(cossim)[::-1])  #Valor de similitud
        similarity_idx_ls.append(np.argsort(cossim)[::-1])  #Indice de Documento

    #print("similarity_ls ordenado:\n")

    return idxs_aleatorios, etiquetas_originales, similarity_idx_ls

In [385]:
idxs_aleatorios, etiquetas_originales, similarity_idx_ls = documentos_aleatorios()

In [386]:
#Función para imprimir documentos originales y sus similares.
def documento_original(doc_ori):
    print(f'Documento Original: {idxs_aleatorios[doc_ori]} \nEtiqueta: {etiquetas_originales[doc_ori]}')
    print(f'\n{30*"-"} Inicio del documento {30*"-"}')
    print(data_train.data[idxs_aleatorios[doc_ori]])    
    print(f'{30*"-"} Fin del documento {30*"-"}')


In [387]:
def lista_documentos_similares(doc_ori):
    print('\tLos documentos con mayor similitud son:\n')
    for idx in similarity_idx_ls[doc_ori][1:6]:
        print(f'\t\t{idx} con etiqueta {data_train.target_names[y_train[idx]]}')  

In [388]:
#Función para imprimir los documentos similares:
def documento_similar (doc_ori, doc_sim):
    documento=similarity_idx_ls[doc_ori][doc_sim]
    print(f'Documento Similar: {documento} \nEtiqueta: {data_train.target_names[y_train[documento]]}\n')
    print(f'{30*"-"} Inicio del documento {30*"-"}')
    print(data_train.data[documento])
    print(f'{30*"-"} Fin del documento {30*"-"}')

##### **Documento 1**

In [389]:
documento_original(0)

Documento Original: 2494 
Etiqueta: sci.crypt

------------------------------ Inicio del documento ------------------------------


I think this to be inaccurate. One can buy TEMPEST equipment commercially.
Even Macs.

David
------------------------------ Fin del documento ------------------------------


In [390]:
lista_documentos_similares(0)

	Los documentos con mayor similitud son:

		4604 con etiqueta sci.crypt
		8512 con etiqueta sci.crypt
		1919 con etiqueta rec.sport.hockey
		10648 con etiqueta sci.crypt
		6417 con etiqueta comp.sys.mac.hardware


**Documentos similares:**

In [391]:
documento_similar(0,1)

Documento Similar: 4604 
Etiqueta: sci.crypt

------------------------------ Inicio del documento ------------------------------

:In article <Apr18.194927.17048@yuma.ACNS.ColoState.EDU>

:>Note that measures to protect yourself from
:>TEMPEST surveillance are still classified, as far as I know.

:I think this to be inaccurate. One can buy TEMPEST equipment commercially.
:Even Macs.

Sure you can buy a TEMPEST approved Mac -- if you have enough
money.  I haven't had any reason to look at this type of pricing
for about 10 years, but a TEMPEST rating in 1982 would raise the
price of a $2,495.00 Radio Shack Model III to something around
$15,000.00.

Bill
------------------------------ Fin del documento ------------------------------


Análisis: 

Es lógico que el algoritmo haya encontrado el documento 4604 como similar al original 2494. Ambos tratan sobre la compra de equipos TEMPEST y mencionan que incluso las computadoras Mac pueden conseguirse con esa clasificación. Además, hay frases muy parecidas, como "One can buy TEMPEST equipment commercially. Even Macs," lo que realmente resalta la conexión entre el contenido y el tema de ambos documentos.

In [392]:
documento_similar(0,2)

Documento Similar: 8512 
Etiqueta: sci.crypt

------------------------------ Inicio del documento ------------------------------

Note that TEMPEST is the name of the shielding standard.  TEMPEST is not
the name of the surveillance technique.
------------------------------ Fin del documento ------------------------------


Análisis: 

El documento 8512 tiene similitudes temáticas con el original 2494, ya que también aborda el tema de TEMPEST, pero las coincidencias son menos directas que en el caso anterior. En este, se aclara una distinción técnica: TEMPEST se refiere al estándar de protección y no a la técnica de vigilancia en sí, mientras que en el documento original se habla más sobre la disponibilidad comercial del equipo TEMPEST.

In [393]:
documento_similar(0,3)

Documento Similar: 1919 
Etiqueta: rec.sport.hockey

------------------------------ Inicio del documento ------------------------------
David



------------------------------ Fin del documento ------------------------------


Análisis:

El documento 1919, etiquetado como rec.sport.hockey, solo contiene la palabra "David", por lo que no tiene ninguna similitud real con el documento original, que está en la categoría sci.crypt. Ambos documentos tratan de cosas muy diferentes; el original habla de equipos TEMPEST en un contexto de seguridad y criptografía, mientras que el documento 1919 parece estar relacionado con deportes, específicamente hockey, sin ninguna conexión entre estos temas. La única coincidencia textual es la palabra "David", lo cual no aporta ningún contenido relevante en común. Es probable que el algoritmo haya seleccionado este documento solo por la coincidencia del nombre, pero claramente no se trata de una similitud real.


In [394]:
documento_similar(0,4)

Documento Similar: 10648 
Etiqueta: sci.crypt

------------------------------ Inicio del documento ------------------------------
[...]

I don't know about classified, but I do seem to remember that unless
you're authorized by the Govt, it's illegal to TEMPEST-shield your
equipment.  Besides, effective TEMPEST-shielding is much more
difficult than you might think (hi Jim!).

	RA

rogue@cs.neu.edu (Rogue Agent/SoD!)
------------------------------ Fin del documento ------------------------------


Análisis: 

El documento 10648 tiene una conexión clara con el original: ambos hablan de TEMPEST, aunque el enfoque varía ligeramente. Mientras que el original menciona la compra de equipos TEMPEST, el 10648 habla sobre las restricciones legales y la dificultad de hacer un blindaje efectivo. Al estar en la misma categoría de seguridad y criptografía (sci.crypt), comparten tanto el tema como un tono similar sobre las complejidades de estos equipos, lo que los hace razonablemente parecidos.

In [395]:
documento_similar(0,5)

Documento Similar: 6417 
Etiqueta: comp.sys.mac.hardware

------------------------------ Inicio del documento ------------------------------
for a
identifies
that

Is this software available either commercially or public domain? If so
where?
------------------------------ Fin del documento ------------------------------


Análisis: 

El documento 6417, etiquetado como comp.sys.mac.hardware, es bastante diferente del original 2494. Mientras que el original trata sobre equipos TEMPEST y temas de seguridad, el 6417 se centra en una consulta sobre la disponibilidad de un software, sin referencias a TEMPEST ni al contexto de seguridad. Aunque ambos mencionan computadoras Mac, el enfoque de los temas es completamente distinto, lo que hace que la similitud entre ellos sea mínima.

##### **Documento 2**

In [396]:
documento_original(1)

Documento Original: 2207 
Etiqueta: rec.sport.baseball

------------------------------ Inicio del documento ------------------------------
And then cjkuo@symantec.com (Jimmy Kuo) quoth:

Well, either way, the Reds have to play a man down for 3 days.

------------------------------ Fin del documento ------------------------------


**Documentos similares**

In [397]:
lista_documentos_similares(1)

	Los documentos con mayor similitud son:

		9044 con etiqueta comp.os.ms-windows.misc
		6848 con etiqueta rec.sport.baseball
		5149 con etiqueta rec.sport.baseball
		6370 con etiqueta comp.sys.ibm.pc.hardware
		2015 con etiqueta rec.sport.baseball


In [398]:
#Documetos similares:
documento_similar(1,1)

Documento Similar: 9044 
Etiqueta: comp.os.ms-windows.misc

------------------------------ Inicio del documento ------------------------------


Try postmaster@norton.com 
or postmaster@symantec.com
--
------------------------------ Fin del documento ------------------------------


Análisis: 

El documento 9044 tiene pocas similitudes reales con el original. Aunque ambos mencionan correos electrónicos y hacen referencia a Symantec, el original trata sobre béisbol y la situación de los Reds, mientras que 9044 parece centrarse en soporte técnico. Las coincidencias son superficiales, y la similitud detectada no refleja una conexión temática significativa entre los textos.

In [399]:
documento_similar(1,2)

Documento Similar: 6848 
Etiqueta: rec.sport.baseball

------------------------------ Inicio del documento ------------------------------

its
plate.
may

Well, there's a big difference between "just not swinging the bat well" and
what the Reds are doing at the plate....UUUGGGHHHH!!!!!

  A Reds fan on the verge of a nervous breakdown....

  --Mike

  "Why is it that the prognosis on Kevin Mitchell is 'Out 2-3 days' no matter
    what day you read it???"
                            -Gary Burbank

------------------------------------------------------------------------
  I've told you before and I'll tell you again.  The strong
   survive and the weak disappear.  We do not intend
   to disappear.

        ---Jimmy Hoffa

                  mtissand@magnus.acs.ohio-state.edu
------------------------------------------------------------------------

------------------------------ Fin del documento ------------------------------


Análisis:

El documento 6848 es bastante similar al original, ya que ambos se centran en el béisbol, mencionan al equipo de los Reds y comparten una etiqueta temática en común. La frustración del aficionado en el segundo documento sobre el desempeño de los Reds refuerza aún más esta conexión en contenido y tono. Por ello, el algoritmo los identificó como altamente similares debido a su tema y emociones compartidas hacia el equipo.

In [400]:
documento_similar(1,3)

Documento Similar: 5149 
Etiqueta: rec.sport.baseball

------------------------------ Inicio del documento ------------------------------

Judging by the way the Reds' pitchers have performed thus far, it
appears to me that the zone has been squeezed to the size of a grape.  

1/2 :-)
------------------------------ Fin del documento ------------------------------


Análisis: 

Este texto se enfoca en el rendimiento de los lanzadores de los Reds y en la percepción de que la zona de strike se ha reducido drásticamente, lo que ofrece una crítica divertida sobre el equipo. La temática del rendimiento del equipo y el tono deportivo coinciden bastante con el original, donde también se menciona una dificultad temporal para los Reds.

In [401]:
documento_similar(1,4)

Documento Similar: 6370 
Etiqueta: comp.sys.ibm.pc.hardware

------------------------------ Inicio del documento ------------------------------
Hi! I was wondering if anyone out there could point me to where I can
get the VESA specifications or any relevant books on this subject.

Regards
Jimmy

------------------------------ Fin del documento ------------------------------


Análisis: 

El documento 6370 tiene muy poca relación con el documento original, ya que aborda una consulta técnica sobre especificaciones VESA en el contexto de hardware de PC, mientras que el documento original trata sobre el equipo de béisbol de los Reds. Es probable que el algoritmo haya detectado coincidencias superficiales, como el uso de un saludo y la firma con un nombre (Jimmy en uno y David en otro), pero estas similitudes no aportan un vínculo significativo en tema ni contenido. En resumen, cualquier conexión entre los dos textos es mínima y sin relevancia temática.

In [402]:
documento_similar(1,5)

Documento Similar: 2015 
Etiqueta: rec.sport.baseball

------------------------------ Inicio del documento ------------------------------
And then cs1442aq@news.uta.edu (cs1442aq) quoth:

That's too bad.  I really had hoped Nolan could end his career with a great
year.  I suppose there is still hope.

------------------------------ Fin del documento ------------------------------


Análisis: 

El documento 2015 guarda una similitud notable con el documento original, ya que ambos están etiquetados como rec.sport.baseball y mantienen un tono de conversación sobre el rendimiento de jugadores en la temporada. En el original se comenta la situación del equipo Reds, mientras que en 2015 se expresa una reflexión sobre el jugador Nolan y sus posibilidades de cerrar su carrera con éxito. Esta coincidencia en el contexto de deportes y en el estilo de conversación sobre jugadores y expectativas deportivas hace que ambos documentos tengan una conexión temática clara.

##### **Documento 3**

In [403]:
documento_original(2)

Documento Original: 8883 
Etiqueta: misc.forsale

------------------------------ Inicio del documento ------------------------------
Hi,

	I have a Quantum ProDrive LPS 40 MB SCSI hard drive for sale.
	It came with my MacIIsi and was replaced by a larger hard drive.
	In great working condition. Fast and quiet. Never had a problem.

	Asking $100+COD shipping or reasonable offer.

	Also for sale with the drive:

	Brand new mounting bracket for MacII or MacSE. It also includes
	SCSI data and power cable. $10 with the HD.

	Please reply with email or call (217)337-5710 and leave message.

	Thanks.

Ding-Kai Chen
dcg6759@uxa.cso.uiuc.edu
------------------------------ Fin del documento ------------------------------


**Documentos similares**

In [404]:
lista_documentos_similares(2)

	Los documentos con mayor similitud son:

		3725 con etiqueta comp.sys.ibm.pc.hardware
		6727 con etiqueta comp.sys.mac.hardware
		7722 con etiqueta comp.sys.ibm.pc.hardware
		4882 con etiqueta comp.sys.ibm.pc.hardware
		1913 con etiqueta comp.sys.ibm.pc.hardware


In [405]:
#Documetos similares:
documento_similar(2,1)

Documento Similar: 3725 
Etiqueta: comp.sys.ibm.pc.hardware

------------------------------ Inicio del documento ------------------------------
I have a 486sx25 computer with a 105 Mg Seagate IDE drive and a controler  
built into the motherboard. I want to add a SCSI drive (a quantum prodrive  
425F 425 MG formatted). I have no documentation at all and I need your  
help!

As I understand it, here is the process of adding such a drive.  Could you  
please tell me if I'm right..

1- Buy a SCSI contoler.  Which one? I know Adaptec is good, but they are  
kind of expensive.  Are there any good boards in the $100 region? I want  
it to be compatible with OS2 and Unix if possible.  Also, I have seen on  
the net that there are SCSI and SCSI2 drives. Is this true? Does the  
adapter need to be the same as the drive? What type of drive is the  
quantum?

2- connect the drive to the adapter via a SCSI cable and the power cable.
Do i have to worry about the power supply? I think I have 200 wat

Al comparar el documento original 8883 con el similar 3725, ambos abordan discos duros SCSI, pero tienen enfoques diferentes. El original se centra en la venta de un disco para Macintosh, mientras que el segundo busca ayuda técnica para instalar un disco en una computadora IBM. Aunque comparten el tema del hardware, su contenido y propósito son distintos, lo que hace que la similitud sea más superficial.

In [406]:
#Documetos similares:
documento_similar(2,2)

Documento Similar: 6727 
Etiqueta: comp.sys.mac.hardware

------------------------------ Inicio del documento ------------------------------
Yes, it is possible to add a second hard drive to a mac IIcx internally. This
is definitely not a recommended procedure by Apple but I have done the equivalent to my CX after upgrading it to a Quadra 700. The power supply is still the
stock CX's and it was able to power two Quantum LPS drives, a PLI SCSI 
accelerator, a Micron technologies 24 bit video board, Daystar QuickCache,
New Technologies Overdrive and 20 meg of RAM simultaneously.

I added a new mount for the drive by attaching angle brackets to the drive
tower. The internal SCSI cable was changed to a longer flat ribbon cable onto
which I added an extra connector about midway. The final HD was internally
terminated and the drive between the motherboard and final HD had its terminator resistor packs removed. Cooling has not been a problem and no SCSI problems
occur with either the PLI SCSI

Al comparar el documento original 8883 con el similar 6727, ambos tratan sobre discos duros para Macintosh, pero con enfoques distintos. El original ofrece un disco duro a la venta, mientras que el segundo describe cómo agregar un segundo disco duro a un Mac IIcx. Aunque comparten el tema de los discos SCSI, el primero se centra en la venta y el segundo en la modificación técnica, lo que sugiere que, a pesar de la conexión temática, su contenido es bastante diferente.

In [407]:
#Documetos similares:
documento_similar(2,3)

Documento Similar: 7722 
Etiqueta: comp.sys.ibm.pc.hardware

------------------------------ Inicio del documento ------------------------------
: >>point of view, why does SCSI have an advantage when it comes to multi-
: >>tasking?  Data is data, and it could be anywhere on the drive.  Can
: >>SCSI find it faster?  can it get it off the drive and into the computer
: >>faster?  Does it have a better cache system?  I thought SCSI was good at
: >>managing a data bus when multiple devices are attached.  If we are
: >>only talking about a single drive, explain why SCSI is inherently
: >>faster at managing data from a hard drive.

: >IDE:  Integrated Device Electronics 
: > currently the most common standard, and is mainly used for medium sized 
: > drives. Can have more than one hard drive. Asynchronous Transfer: ~5MB/s max.

: Why don't you start with the spec-sheet of the ISA bus first?
: You can quote SCSI specs till you're blue in the face, but if they
: exceed the ISA bus capability, t

Al comparar el documento original 8883 con el similar 7722, se nota que ambos tocan temas relacionados con SCSI, pero desde perspectivas muy diferentes. El original ofrece un disco duro SCSI a la venta, mientras que el segundo se sumerge en una discusión técnica sobre las ventajas del SCSI en multitarea y su rendimiento en comparación con IDE. Aunque comparten el término "SCSI", el enfoque y el contenido son bastante distintos, lo que refleja que, a pesar de la conexión superficial, no son realmente similares en su esencia.

In [408]:
#Documetos similares:
documento_similar(2,4)

Documento Similar: 4882 
Etiqueta: comp.sys.ibm.pc.hardware

------------------------------ Inicio del documento ------------------------------
Wow, you guys are really going wild on this IDE vs. SCSI thing, and I think
it's great!

However, I think that some people (such as myself) would benefit from answers
to the simple(?) question:  Which would YOU choose, and why?

Like lots of people, I'd really like to increase my data transfer rate from
the hard drive.  Right now I have a 15ms 210Mb IDE drive (Seagate 1239A), and
what I would say is a standard (not special, no cache I believe) IDE controller
card on my ISA 486-50.

I'm currently thinking about adding another HD, in the 300Mb to 500Mb range.
And I'm thinking hard (you should hear those gears a-grinding in my head)
about buying a SCSI drive (SCSI for the future benefit).  I believe I'm getting
something like 890Kb/sec transfer right now (according to NU).

How would this
number compare if I bought the state-of-the-art SCSI card f

Al comparar el documento original 8883 con el similar 4882, se observa que ambos abordan el tema de almacenamiento, pero desde ángulos diferentes. El original se centra en la venta de un disco duro SCSI, mientras que el segundo documento se enfoca en la discusión sobre las ventajas de SCSI frente a IDE, especialmente en términos de tasas de transferencia de datos. Aunque ambos documentos mencionan SCSI, el enfoque práctico de la venta en el original contrasta con la búsqueda de consejos sobre rendimiento en el segundo. En resumen, aunque están relacionados con el almacenamiento, los contextos y propósitos son distintos.

In [409]:
#Documetos similares:
documento_similar(2,5)

Documento Similar: 1913 
Etiqueta: comp.sys.ibm.pc.hardware

------------------------------ Inicio del documento ------------------------------
From article <1qq7i1INNdqc@dns1.NMSU.Edu>, by bgrubb@dante.nmsu.edu (GRUBB):

[Tons of stuff deleted on SCSI vs. IDE question...]

Wow, you guys are really going wild on this IDE vs. SCSI thing, and I think
it's great!

Like lots of people, I'd really like to increase my data transfer rate from
the hard drive.  Right now I have a 15ms 210Mb IDE drive (Seagate 1239A), and
what I would say is a standard (not special, no cache I believe) IDE controller
card on my ISA 486-50.

I'm currently thinking about adding another HD, in the 300Mb to 500Mb range.
And I'm thinking hard (you should hear those gears a-grinding in my head)
about buying a SCSI drive (SCSI for the future benefit).  I believe I'm getting
something like 890Kb/sec transfer right now (according to NU).  How would this
number compare if I bought the state-of-the-art SCSI card for my ISA

Al comparar el documento original 8883 con el similar 1913, ambos tocan el tema del almacenamiento de datos, específicamente sobre SCSI e IDE. El primero se centra en la venta de un disco duro SCSI, mientras que el segundo plantea dudas sobre el rendimiento de ambos sistemas y busca consejos para mejorar la velocidad de transferencia al considerar un nuevo disco duro. Aunque comparten el mismo interés por el almacenamiento, el original es una oferta concreta, mientras que el segundo es más bien una solicitud de orientación. En esencia, hay un tema común, pero los enfoques son bastante distintos.

##### **Documento 4**

In [410]:
documento_original(3)

Documento Original: 8003 
Etiqueta: talk.politics.misc

------------------------------ Inicio del documento ------------------------------

Agreed.  


It is a failure of libertarianism if the ideology does not provide any
reasonable way to restrain such actions other than utopian dreams.  Just
as Marxism "fails" to specify how pure communism is to be achieved and
the state is to "wither away," libertarians frequently fail to show how
weakening the power of the state will result in improvement in the human
condition.


This is a strawman argument and fails on several grounds.  In this case,
"limited" and "big" government are not defined.  I would point out that
Lebanon, Somalia, and the former Yugoslavia are by some definitions nations
with "limited" government, while the US, Canada, and nations in Western 
Europe (where "people would rather live") are often pointed out as 
nations with "big government" from a libertarian point of view.  

The argument is not between those who want "li

**Documentos similares**

In [411]:
lista_documentos_similares(3)

	Los documentos con mayor similitud son:

		670 con etiqueta talk.politics.misc
		4072 con etiqueta talk.politics.misc
		3938 con etiqueta sci.crypt
		6602 con etiqueta talk.politics.misc
		7200 con etiqueta talk.politics.misc


In [412]:
#Documetos similares:
documento_similar(3,1)

Documento Similar: 670 
Etiqueta: talk.politics.misc

------------------------------ Inicio del documento ------------------------------

I'm afraid that I've lost the thread here.  I didn't suggest that all 
government regulations be subject to referenda.  So I don't follow the 
comments above.


I mean that an ideology that treats all government regulation as equally
undesirable and seeks to abolish all regulations is unlikely to draw
support among more than a miniscule portion of the electorate.

Furthermore, I am suggesting that such a plan is not feasible in an
industrial society because the weight of litigation and/or misery it
would produce would effectively crush productive effort.

No, I'm not surprised.  I just think it's interesting that on one hand
libertarians assume a limited government can be decreed, yet on the other
posit an entire government made up of people who carry guns.  (I realize
that many libertarians assume that such a government will be 
counterbalanced by a

Al comparar el documento original 8003 con el documento similar 670, ambos comparten un enfoque en la discusión sobre la ideología política y el papel del gobierno. El documento original critica el libertarianismo por su falta de propuestas claras para mejorar la condición humana al debilitar el estado, mientras que el documento 670 cuestiona la viabilidad de abolir todas las regulaciones gubernamentales, sugiriendo que eso no tendría mucho apoyo popular. Ambos textos reflejan preocupaciones sobre el equilibrio entre la libertad individual y la necesidad de un gobierno regulador, aunque cada uno aborda el tema desde perspectivas ligeramente diferentes. En resumen, hay una conexión en su discusión sobre el gobierno, pero cada uno enfatiza distintos aspectos de la ideología política.

In [413]:
#Documetos similares:
documento_similar(3,2)

Documento Similar: 4072 
Etiqueta: talk.politics.misc

------------------------------ Inicio del documento ------------------------------

Of course, one again faces the question of how one circumscribes government
power (and keeps it circumscribed) in a complex society when it is in the 
interest of neither capitalists nor consumers to refrain from using 
government power for their own ends.  But apart from that little 
conundrum...

It would seem that a society with a "failed" government would be an ideal
setting for libertarian ideals to be implemented.  Now why do you suppose
that never seems to occur?...

I wouldn't call it "vague."  I'd call it elastic.  All "regulation" is 
not necessarily the same.  By opposing all government regulation, some 
libertarians treat every system from a command economy to those that
regulate relatively free markets as identical.  That's one reason
many of the rest of us find their analysis to be simplistic. 

jsh
------------------------------ Fin d

Al comparar el documento original 8003 con el documento similar 4072, ambos discuten el papel del gobierno y la ideología libertaria, pero desde ángulos distintos. El documento original critica la falta de claridad en las propuestas libertarias para mejorar la sociedad, mientras que el documento 4072 plantea dudas sobre cómo limitar el poder del gobierno en una sociedad compleja y sugiere que un gobierno "fallido" podría ser el escenario ideal para aplicar ideales libertarios, lo cual nunca sucede. Ambos textos destacan las dificultades de equilibrar la regulación gubernamental y las libertades individuales, pero el segundo se centra más en la crítica de la simplificación de los argumentos libertarios. En resumen, ambos comparten preocupaciones sobre el gobierno y la regulación, pero abordan la discusión desde perspectivas diferentes.

In [414]:
#Documetos similares:
documento_similar(3,3)

Documento Similar: 3938 
Etiqueta: sci.crypt

------------------------------ Inicio del documento ------------------------------

The willingness of the majority of the people to give up their freedom in 
exchange for a sense of safety is hardly limited to the USA, and is an 
endemic problem in any human society of any appreciable size.  The structure 
of the US government does try to combat this tendency to some extent, but 
fighting entropy is always a losing battle.  Most people would rather have 
comfort than freedom.  The paradox is that you can't really have the former, 
in the long term, unless you have the latter.

One of the reasons that I probably come across to some people as a weird 
cross between a libertarian and an "establishment tool" is that I end up 
taking an utterly pragmatic view of government.  I don't get up in arms when 
the government fails to protect the interests of the people, because in my 
lifetime it never has--therefore, I have no expectation that it wil

El documento original 8003 y el documento similar 3938 se consideran similares porque ambos discuten la tensión entre la libertad individual y la intervención gubernamental. Mientras el original critica la falta de soluciones claras en el libertarianismo para mejorar la condición humana, el segundo enfatiza la tendencia de las personas a sacrificar su libertad por seguridad. Ambos abordan cómo los individuos manejan su relación con el gobierno, aunque desde perspectivas diferentes.

In [415]:
#Documetos similares:
documento_similar(3,4)

Documento Similar: 6602 
Etiqueta: talk.politics.misc

------------------------------ Inicio del documento ------------------------------

[...]


You seem to be saying that a LIMITED government will provide MORE
opportunities for private interests to use it to pursue their own
agendas, and asking libertarians to prove that this will NOT happen.
While I can't offer such a proof, it seems pretty damn plausible that
if the government does not regulate a particular area, it cannot become
a tool of private interests to pursue their own agendas in that area.
I rather suspect that it's the sort of government we have NOW that is
more likely to become such a tool, and that it IS such a tool in many
instances.


I suspect that this is because "improvement in the human condition" as
you define it is not the primary goal of libertarianism, and would not
be the primary goal of a libertarian government.  My impression of
libertarianism is that its primary goal is the elimination of
government coerc

Al comparar el documento original 8883 con el similar 4882, se observa que ambos abordan el tema de almacenamiento, pero desde ángulos diferentes. El original se centra en la venta de un disco duro SCSI, mientras que el segundo documento se enfoca en la discusión sobre las ventajas de SCSI frente a IDE, especialmente en términos de tasas de transferencia de datos. Aunque ambos documentos mencionan SCSI, el enfoque práctico de la venta en el original contrasta con la búsqueda de consejos sobre rendimiento en el segundo. En resumen, aunque están relacionados con el almacenamiento, los contextos y propósitos son distintos.

In [416]:
#Documetos similares:
documento_similar(3,5)

Documento Similar: 7200 
Etiqueta: talk.politics.misc

------------------------------ Inicio del documento ------------------------------

If we had backed him strongly early on I doubt there would be the
problem there is now.  Many Russians became disillusioned with democracy
and reforms when they felt, rightly IMO, that the West didn't care.

Yeltsin was virtually promised massive aid (once Bush got over his
Gorby-mania.)  This probably kept him from dismantling the congress
and calling for new elections.  Well, the aid didn't come through
and we didn't make sure it went to the proper places and now the
anti-reformers are gaining strength where before they were in
hiding.


If they don't reform I don't believe in giving them money.  However,
I think this is too important to take a non-interventionist approach.
This is what really bugs me about Libertarianism -- it sounds like
'it'll all be the same in a hundred years time.  no need to do
anything.'

Despite the wishes of Libertarians

Al comparar el documento original 8883 con el similar 1913, ambos tocan el tema del almacenamiento de datos, específicamente sobre SCSI e IDE. El primero se centra en la venta de un disco duro SCSI, mientras que el segundo plantea dudas sobre el rendimiento de ambos sistemas y busca consejos para mejorar la velocidad de transferencia al considerar un nuevo disco duro. Aunque comparten el mismo interés por el almacenamiento, el original es una oferta concreta, mientras que el segundo es más bien una solicitud de orientación. En esencia, hay un tema común, pero los enfoques son bastante distintos.

##### **Documento 5**

In [417]:
documento_original(4)

Documento Original: 2581 
Etiqueta: talk.politics.misc

------------------------------ Inicio del documento ------------------------------
Government-Mandated Energy Conservation is Unnecessary and Wastful, Study Finds

  Washington, DC -- The energy tax and subsidized energy-efficiency
  measures supported by President Clinton and Energy Secretary Hazel
  O'Leary are based on faulty assumptions, a new study from the Cato
  Institute points out.

    According to Jerry Taylor, Cato's director of natural resource studies,
  we are not running out of sources of energy. The world now has almost 10
  times the proven oil reserves it had in 1950 and twice the reserves of
  1970. Proven reserves of coal and natural gas have increased just as
  dramatically.

    When standards of living, population densities, and industrial
  structures are controlled for, the United States is no less energy
  efficient than Japan and more energy efficient than many of the Group
  of Seven nations.

    Ener

**Documentos similares**

In [418]:
lista_documentos_similares(4)

	Los documentos con mayor similitud son:

		6421 con etiqueta talk.politics.misc
		7970 con etiqueta sci.electronics
		3746 con etiqueta talk.politics.misc
		7441 con etiqueta talk.politics.guns
		5423 con etiqueta talk.politics.mideast


In [419]:
#Documetos similares:
documento_similar(4,1)

Documento Similar: 6421 
Etiqueta: talk.politics.misc

------------------------------ Inicio del documento ------------------------------


While I agree with much of this post, one point seems mis-directed...


And when controlled for usage of oil, gas, etc. energy efficiency in
all countries turns out to be identical :-)

To take population density as an example, one way to reduce energy
used in transportation is surely to concentrate the population in
dense urban areas (though this might, of course, have other
disadvantages, possibly even relating to energy use).  The fact that
Japan is forced to do this by the nature of the country, while the US
is not, does not mean that people in the US would be unable to do this
if given sufficient motive to conserve energy.
------------------------------ Fin del documento ------------------------------


Tiene sentido que el algoritmo haya detectado similitudes. Ambos documentos abordan la eficiencia energética, comparando a EE. UU. con otros países y discutiendo factores como la densidad poblacional y las condiciones que afectan el uso de energía. Aunque el documento 6421 es más breve y conversa una idea diferente, comparte el enfoque en factores contextuales de la eficiencia energética, lo cual justifica la relación.

In [420]:
#Documetos similares:
documento_similar(4,2)

Documento Similar: 7970 
Etiqueta: sci.electronics

------------------------------ Inicio del documento ------------------------------

try sci.energy
------------------------------ Fin del documento ------------------------------


El documento 7970 no realmente se compara con el documento Original (2581) simplemente sugiere cambiar a un tema diferente, "sci.energy", sin aportar ideas o argumentos sobre la conservación de energía. No aborda el contenido del original, creo que no tiene sentido considerarlo un documento similar en ese contexto. Es como si alguien entrara a una conversación sobre un tema específico y solo dijera que deberíamos hablar de otra cosa; no hay conexión entre ambos.

In [421]:
#Documetos similares:
documento_similar(4,3)

Documento Similar: 3746 
Etiqueta: talk.politics.misc

------------------------------ Inicio del documento ------------------------------
THE WHITE HOUSE

                  Office of the Press Secretary
                 (Vancouver, British Columbia) 
______________________________________________________________


                       BACKGROUND BRIEFING
                               BY
                 SENIOR ADMINISTRATION OFFICIALS


                          April 4, 1993
	     
                          Canada Place
                  Vancouver, British Columbia  


9:40 A.M. PST
	     
	     
	     Folks, we're about to start the BACKGROUND BRIEFING 
on the aid package.

	     SENIOR ADMINISTRATION OFFICIAL:  Good morning.  The 
President -- President Clinton and President Yeltsin agreed 
yesterday on a series of American initiatives to support economic 
and political reform in Russia, and it's valued at $1.6 billion.  

	     Before taking your questions and running through th

El Documento Similar 3746 no tiene relación con el Documento Original. Este último se centra en la conservación de energía y las políticas asociadas, mientras que el documento similar es una transcripción de una conferencia de prensa de la Casa Blanca. Aunque ambos documentos provienen de contextos políticos, el enfoque y el contenido son completamente diferentes. Es como comparar un artículo de opinión sobre eficiencia energética con una minuta de una reunión; son temas distintos y no se pueden relacionar directamente.

In [422]:
#Documetos similares:
documento_similar(4,4)

Documento Similar: 7441 
Etiqueta: talk.politics.guns

------------------------------ Inicio del documento ------------------------------
Posted by Cathy Smith for L. Neil Smith

                           WEIRD SCIENCE

Everyone knows how to tell when a politician is lying:  his lips 
move.  What may not be equally obvious is that there are 
politicians and then there are politicians -- and that the phrase 
"political science" is subject to more than one interpretation.  

Years ago, we heard how "scientists" were worried that a new Ice 
Age might be coming, and later on that "nuclear winter" -- smoke 
and dust thrown into the atmosphere by full-scale international 
unpleasantness -- was a possibility.  Something like that may even 
have killed the dinosaurs.  

What we didn't hear was that no actual data supported any of this, 
that real-world events (the burning of Kuwaiti oil fields) tended 
to discredit it, that mostly it was propaganda meant to weaken 
values that made America th

El Documento Similar 7441 no tiene relación con el Documento Original sobre conservación de energía. Este texto se enfoca en la crítica a los políticos y la interpretación de la "ciencia política", con referencias a teorías sobre el clima y otros temas que parecen más orientados a una discusión sobre política y armas, dado su contexto. Mientras que el original aborda cuestiones sobre la eficiencia energética y la política energética, el documento similar se adentra en una crítica más amplia y satírica sobre la política en general. Son temas y enfoques distintos, así que no se pueden comparar directamente.

In [423]:
#Documetos similares:
documento_similar(4,5)

Documento Similar: 5423 
Etiqueta: talk.politics.mideast

------------------------------ Inicio del documento ------------------------------


Anytime.


Suffering from a severe case of myopia? No Muslim left alive - not a 
single one. Leading the first Armenian units who crossed the Ottoman 
border in the company of the Russian invaders was the former Ottoman 
Parliamentary representative for Erzurum, Karekin Pastirmaciyan, who 
assumed the revolutionary name Armen Garo. Another former Ottoman 
parliamentarian, Hamparsum Boyaciyan, led the Armenian guerrilla forces 
who ravaged Turkish villages behind the lines under the nickname "Murad", 
especially ordering that 

         'Kill Turks and Kurds wherever you find them and in 
          whatever circumstances you find them. Turkish children 
          also should be killed as they form a danger to the 
          Armenian nation.' (Hamparsum Boyadjian - 1914)[1]

 [1] M. Varandian, "History of the Dashnaktsutiun," p. 85.

Another forme

El Documento Similar 5423 no tiene relación con el Documento Original sobre conservación de energía. Este texto se centra en un conflicto histórico entre armenios y otomanos, incluyendo referencias a violencia y figuras políticas de esa época. Por otro lado, el documento original trata sobre políticas energéticas y eficiencia. Es como comparar manzanas con naranjas; aunque ambos son temas políticos, no tienen nada en común en términos de contenido o enfoque. Por lo tanto, no hay una conexión significativa entre ellos.

### **2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámetros instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.


In [424]:
#Instanciar el vectorizador 2, con parametros ajustados
vectorizador_2 = TfidfVectorizer(
    max_features=1000,         # Limitar a las 1000 características más frecuentes
    stop_words='english',      # Ignorar palabras comunes en inglés
    ngram_range=(1, 2),       # Considerar unigramas y bigramas 
    min_df=2,                  # Incluir palabras que aparecen en al menos 2 documentos
    max_df=0.95,               # Excluir palabras que aparecen en más del 95% de los documentos
    norm='l2',                 # Normalizar usando la norma L2
    smooth_idf=True            # Suavizar el IDF
)


Ajustamos los sets con el vectorizador_2 con parametros ajustados:

In [425]:
#Sets de entrenamiento
X_train = vectorizador_2.fit_transform(data_train.data)
y_train = data_train.target

#Sets de testeo
X_test = vectorizador_2.transform(data_test.data)
y_test = data_test.target

In [426]:
#Función para buscar los mejores hiperparametros: 

def best_hp_model (model):

    param_grid = {
    'alpha': [0.1, 0.15, 0.20, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
    'fit_prior': [True, False]
    }

    # Instanciar GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro')  # Usar F1-score macro como métrica

    # Ajustar el modelo a los datos
    grid_search.fit(X_train, y_train)

    # Imprimir los mejores parámetros y el mejor score
    print("Mejores parámetros:", grid_search.best_params_)
    #print("Mejor F1-score macro:", grid_search.best_score_)
    
    return grid_search.best_params_

##### Naive Bayer Multinomial

In [427]:
#Primero busco los mejores hiperparametros para el modelo: 
best_hp_MNB = best_hp_model(MultinomialNB())

#Luego instancio el modelo con los parametros devueltos: 
mod_MNB = MultinomialNB(alpha=best_hp_MNB['alpha'], fit_prior=best_hp_MNB['fit_prior'])


Mejores parámetros: {'alpha': 0.25, 'fit_prior': False}


In [428]:
#Entreno el modelo
mod_MNB.fit(X_train, y_train)

#Hago una predicción y calculo el F1-SCORE
y_pred_MNB =  mod_MNB.predict(X_test)
f1_MNB = f1_score(y_test, y_pred_MNB, average='macro')
print(f'F1-Score de NB Multinomial:\n\t{f1_MNB}')

F1-Score de NB Multinomial:
	0.5181612647791983


##### Naive Bayer Complement

In [429]:
#Primero busco los mejores hiperparametros para el modelo: 
best_hp_CNB = best_hp_model(ComplementNB())

#Luego instancio el modelo con los parametros devueltos: 
mod_CNB = ComplementNB(alpha=best_hp_CNB['alpha'], fit_prior=best_hp_CNB['fit_prior'])

Mejores parámetros: {'alpha': 10.0, 'fit_prior': True}


In [430]:
#Entreno el modelo
mod_CNB.fit(X_train, y_train)

#Hago una predicción y calculo el F1-SCORE
y_pred_CNB =  mod_CNB.predict(X_test)
f1_CNB = f1_score(y_test, y_pred_CNB, average='macro')
print(f'F1-Score de NB Complemento:\n\t{f1_CNB}')

F1-Score de NB Complemento:
	0.4826235110703959


### 3. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".

In [431]:
#La matriz a trasponer es nuestra X_train que es del tipo: <class 'scipy.sparse._csr.csr_matrix'>
# Transponer la matriz dispersa
X_train_transposed = X_train.transpose()

In [432]:
# Convertir la matriz transpuesta a un DataFrame
df_palabras = pd.DataFrame(X_train_transposed.toarray(), index=vectorizador_2.get_feature_names_out())


In [435]:
# Lista de palabras de interés
palabras_interes = ['people', 'think', 'make', 'just', 'time']

# Crear un diccionario para almacenar las palabras más similares
palabras_similares = {}

for palabra in palabras_interes:
    # Obtener la fila correspondiente a la palabra
    vector_palabra = df_palabras.loc[palabra].values.reshape(1, -1)

    # Calcular la similitud de coseno de esta palabra con todas las demás
    similitudes = cosine_similarity(vector_palabra, df_palabras)[0]

    # Crear un DataFrame temporal con palabras y sus similitudes
    similitud_df = pd.DataFrame({'palabra': df_palabras.index, 'similitud': similitudes})

    # Ordenar y seleccionar las 5 más similares, excluyendo la palabra original
    similares = similitud_df[similitud_df['palabra'] != palabra].nlargest(5, 'similitud')
    palabras_similares[palabra] = similares['palabra'].tolist()

In [436]:
print('Las palabras similares a las elegidas son: \n')
for palabra, similares in palabras_similares.items():
    print(f"\t{palabra}: {similares}")

Las palabras similares a las elegidas son: 

	people: ['don', 'think', 'just', 'government', 'like']
	think: ['don think', 'don', 'people', 'just', 'like']
	make: ['like', 'don', 'people', 'sure', 'offer']
	just: ['don', 'like', 'people', 'think', 'know']
	time: ['long', 'like', 'don', 'just', 'good']


En el resultado puedo ver que las palabras elegidas tienden a estar relacionadas con temas de conversación y debate, lo cual tiene sentido dado que el dataset fetch_20newsgroups se centra en foros de discusión sobre temas de actualidad.

Observo que algunas palabras como "don", "think", y "like" aparecen como similares a varias de las palabras elegidas, lo que sugiere que son términos centrales en muchas discusiones. Esto me indica que se usan con frecuencia en varios contextos para expresar opiniones o ideas.

Palabras como "government", "know", y "offer" también destacan entre los términos similares, lo cual me indica que hay una tendencia en el dataset a explorar temas de sociedad, autoridad, y compartir conocimiento.

Este análisis muestra que el lenguaje en el dataset refleja un enfoque hacia la interacción social y el debate, lo que es típico en grupos de noticias donde la gente intercambia ideas sobre política, opiniones y temas cotidianos.